<a href="https://colab.research.google.com/github/PrathUpadhyay/NLP/blob/master/Assignment_3_18210058.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess

In [11]:
from google.colab import drive
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import re
import string
import operator
from nltk.util import ngrams
import numpy as np

nltk.download('punkt')

drive.mount("/content/drive")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# read file from google-drive, colab doesnt keeps uploaded files persistently

tweet_file = open(r"drive/My Drive/Colab Notebooks/content/nlp assignment 3/train.txt", encoding="utf8")
tweets = tweet_file.read()

# test file separate
tweets_file_test = open(r"drive/My Drive/Colab Notebooks/content/nlp assignment 3/test.txt", encoding="utf8")
tweets_test = tweets_file_test.read()

# tweets_list = nltk.sent_tokenize(tweets)
tweets_list = re.split('\n\n',tweets)
tweets_list_test = re.split('\n\n',tweets_test)

# last sentence empty string
tweets_list = tweets_list[:-1]
tweets_list_test = tweets_list_test[:-1]

# tweets_list
# len(tweets_list)
# sent_tokenize_list = nltk.sent_tokenize(speeches)

In [13]:
#clean train dataset
tweets_clean_list = []
sentiments_list = []

for sent in tweets_list:
  # remove \n
  temp = re.sub("\n", ' ', sent)

  # remove ’
  temp = re.sub("\’", '', sent)

  # convert to lowercase
  temp = temp.lower()

  # remove unnecessary punctuations
  temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  # remove first two words from the message, its metadata
  temp = temp.split(' ', 3)

  # take the next word as sentiment
  sentiment = temp[2]

  # Retreive only the message
  new_msg = temp[3]

  new_msg = new_msg.replace("hin", "")
  new_msg = new_msg.replace("eng", "")
  new_msg = new_msg.replace("o", "")

  # emoji removal by emoji pattern
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  new_msg = emoji_pattern.sub(r'', new_msg)

  # remove multiple consecutive spaces from message
  new_msg = re.sub(' +', ' ', new_msg)

  #after cleaning up, append to new list
  tweets_clean_list.append(new_msg)
  sentiments_list.append(sentiment)

tweets_clean_list[:10]
sentiments_list[:10]

['negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [14]:
#Clean Test dataset
tweets_clean_list_test = []
sentiments_list_test = []

for sent in tweets_list_test:
  # remove \n
  temp = re.sub("\n", ' ', sent)

  # remove ’
  temp = re.sub("\’", '', sent)

  # convert to lowercase
  temp = temp.lower()

  # remove unnecessary punctuations
  temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  # remove first three words from the message, its metadata
  temp = temp.split(' ', 3)

  # take the next word as sentiment
  sentiment = temp[2]

  # Retreive only the message
  new_msg = temp[3]

  new_msg = new_msg.replace("hin", "")
  new_msg = new_msg.replace("eng", "")
  new_msg = new_msg.replace("o", "")

  # emoji removal by emoji pattern
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  new_msg = emoji_pattern.sub(r'', new_msg)

  # remove multiple consecutive spaces from message
  new_msg = re.sub(' +', ' ', new_msg)

  #after cleaning up, append to new list
  tweets_clean_list_test.append(new_msg)
  sentiments_list_test.append(sentiment)


tweets_clean_list_test[:10]


sentiments_list_test[:10]


['neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative']

# Tokenize data using TF-IDF

In [0]:
# Objects to be used further
# tweets_clean_list
# sentiments_list
# tweets_clean_list_test
# sentiments_list_test

from keras.preprocessing.sequence import pad_sequences

train_dict = {"Message": tweets_clean_list, "Sentiment": sentiments_list}
df = pd.DataFrame(train_dict)

test_dict = {"Message": tweets_clean_list_test, "Sentiment": sentiments_list_test}
test_df = pd.DataFrame(test_dict)

from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7)  
X = tfidfconverter.fit_transform(df['Message']).toarray()


# LSTM

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN, LSTM, TimeDistributed, Embedding

embed_dim = 128
lstm_out = 300

model = Sequential()
model.add(Embedding(3000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, dropout=0.2, recurrent_dropout=0.2)`
  if __name__ == '__main__':


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2000, 128)         384000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               514800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 903       
Total params: 899,703
Trainable params: 899,703
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
Y = pd.get_dummies(df['Sentiment']).values

batch_size = 500

model.fit(X, Y, batch_size=batch_size, nb_epoch = 8,  verbose = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/8
15131/15131 [==============================] - 153s 10ms/step - loss: 1.0955 - acc: 0.3694
Epoch 2/8
15131/15131 [==============================] - 150s 10ms/step - loss: 1.0942 - acc: 0.3700
Epoch 3/8
15131/15131 [==============================] - 153s 10ms/step - loss: 1.0944 - acc: 0.3725
Epoch 4/8
15131/15131 [==============================] - 152s 10ms/step - loss: 1.0943 - acc: 0.3726
Epoch 5/8
15131/15131 [==============================] - 153s 10ms/step - loss: 1.0945 - acc: 0.3726
Epoch 6/8
15131/15131 [==============================] - 152s 10ms/step - loss: 1.0942 - acc: 0.3726
Epoch 7/8
15131/15131 [==============================] - 152s 10ms/step - loss: 1.0944 - acc: 0.3726
Epoch 8/8
15131/15131 [==============================] - 151s 10ms/step - loss: 1.0943 - acc: 0.3726


#Test

In [18]:
# X_test, Y_test = tweets_clean_list_test, sentiments_list_test

X_test = tfidfconverter.transform(tweets_clean_list_test).toarray()
Y_test = pd.get_dummies(test_df['Sentiment']).values


_, accuracy = model.evaluate(X_test, Y_test, batch_size = 200, verbose = 1)

print("Accuracy is ", accuracy)



1869/1869 [==============================] - 10s 5ms/step
Accuracy is  0.4034242909690669


In [19]:

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
y_pred1 = model.predict(X_test)
y_pred = np.argmax(y_pred1, axis=1)
Y_test = np.argmax(Y_test, axis = 1 )

print (y_pred)
# print(len())
# print (Y_test[:10])

[1 1 1 ... 1 1 1]


In [21]:
from sklearn.metrics import classification_report
# print(Y_test)
sentiment_names = ['Positive', 'Neutral', 'Negative']
print(classification_report(Y_test, y_pred, target_names=sentiment_names))

              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00       533
     Neutral       0.40      1.00      0.57       754
    Negative       0.00      0.00      0.00       582

    accuracy                           0.40      1869
   macro avg       0.13      0.33      0.19      1869
weighted avg       0.16      0.40      0.23      1869



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
